In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

这本笔记本是[Sara Robinson和Ivan Chueng](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/matching_engine/sdk_matching_engine_for_indexing.ipynb)的笔记本的修订版本。

# GCP 上的 E2E ML： MLOps 阶段6：服务：使用 Vertex AI Vector Search 入门

<table align="left">

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_matching_engine.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
    <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_matching_engine.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_matching_engine.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用`Vertex AI矢量搜索`服务。这是一个云AI服务，是用于向量（即嵌入）的适当最近邻（ANN）索引和匹配服务，具有高可伸缩性和低延迟。该服务构建在由谷歌研究开发的[近似最近邻（ANN）技术](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html)之上。

有几个级别可以使用这项服务。

*无代码*

在本教程中进行演示。用户使用自己的嵌入进行索引和查询。

*低代码*

用户使用Vertex AI预构建算法Twivel和TwoTowers构建嵌入。

*高代码*

用户使用`Vertex AI按示例解释`配置服务二进制文件，从模型中生成嵌入，进行索引和查询。

了解更多关于[Vertex AI矢量搜索](https://cloud.google.com/vertex-ai/docs/matching-engine/overview)。

### 目标

在本笔记本中，您将学习如何创建近似最近邻居（ANN）索引，并对索引进行查询。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI 矢量搜索`

执行的步骤包括：

- 创建 ANN 索引。
- 创建带有 VPC 网络的 IndexEndpoint
- 部署 ANN 索引
- 执行在线查询
- 部署蛮力（brute force）索引
- 在 ANN 索引和蛮力索引之间执行校准。

嵌入

这个教程中使用的预先构建的嵌入是[GloVe数据集](https://nlp.stanford.edu/projects/glove/)。

“GloVe是一种用于获取单词向量表示的无监督学习算法。 训练是基于语料库中聚合的全局词-词共现统计数据进行的，结果表示展示了词向量空间中有趣的线性子结构。”

成本

本教程使用Google Cloud的计费组件：

- Vertex AI
- 云存储

了解 [Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
价格](https://cloud.google.com/storage/pricing)，并使用 [价格
计算器](https://cloud.google.com/products/calculator/)
根据您预期的使用情况生成成本估算。

## 安装

安装执行该笔记本所需的包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install -U grpcio-tools {USER_FLAG} -q
! pip3 install -U h5py {USER_FLAG} -q

重新启动内核

在安装附加包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU运行时

*如果有此选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行类型 > GPU**

### 设置您的谷歌云项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您会获得$300的免费信用额度用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API，Compute Engine API和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. [启用服务网络API](https://console.cloud.google.com/flows/enableapi?apiid=servicenetworking.googleapis.com)。

5. [启用Cloud DNS API](https://console.cloud.google.com/flows/enableapi?apiid=dns.googleapis.com)。

6. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

7. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter在以`!`为前缀的行上运行为shell命令，并会替换以`$`为前缀的Python变量。

将您的项目ID设置为## 

**如果您不知道您的项目ID**，您可以使用 `gcloud` 获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

获取您的项目编号

现在项目 ID 已设置，您会得到相应的项目编号。

In [ ]:
shell_output = ! gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = shell_output[0]
print("Project Number:", PROJECT_NUMBER)

#### 区域

您还可以更改“REGION”变量，该变量用于整个笔记本中的操作。以下是支持 Vertex AI 的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶来训练 Vertex AI。并非所有区域都支持所有 Vertex AI 服务。

了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在进行在线教程会话，您可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您可以为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称中。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证你的Google Cloud账号

**如果你正在使用Vertex AI Workbench笔记本**，你的环境已经验证通过。跳过这一步。

**如果你正在使用Colab**，运行下面的单元格，并按照提示进行身份验证，通过oAuth。

**否则**，请按照以下步骤操作：

在云控制台中，转到[创建服务账号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账号**。

在**服务账号名称**字段中，输入一个名称，然后点击**创建**。

在**授予此服务账号项目访问权限**部分，点击角色下拉列表。在筛选框中输入“Vertex”，选择**Vertex管理员**。在筛选框中输入“Storage Object Admin”，选择**Storage Object Admin**。

点击创建。包含您密钥的JSON文件将下载到您的本地环境中。

在下面的单元格中，输入服务账号密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用什么笔记本环境，以下步骤都是必须的。**

当您初始化用于 Python 的 Vertex AI SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是所有与您的数据集和模型资源相关的数据在会话之间保留的地方。

在下面设置您的云存储桶的名称。桶的名称必须在所有 Google 云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有你的存储桶尚不存在：运行以下单元格以创建你的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最终，通过检查云存储桶中的内容来验证对其的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
import h5py

初始化Vertex AI Python SDK

为您的项目和相应的存储桶初始化Vertex AI Python SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### 下载并准备预构建的GloVe嵌入

GloVe嵌入包含一组预训练的嵌入。这些嵌入被分为“训练”和“测试”两部分。
您可以从“训练”部分创建一个`Vertex AI Vector Search`索引，并使用“测试”部分中的嵌入向量作为查询向量来测试该索引。

*注意:* 尽管数据分割使用了术语“训练”，但这些是预训练的嵌入，因此可以立即用于索引搜索。术语“训练”和“测试”拆分仅用于与常规机器学习术语保持一致。

In [ ]:
! gsutil cp gs://cloud-samples-data/vertex-ai/matching_engine/glove-100-angular.hdf5 .

将嵌入加载到内存中
从HDF5存储格式中加载GloVe嵌入到内存中。

In [ ]:
h5 = h5py.File("glove-100-angular.hdf5", "r")
train = h5["train"]
test = h5["test"]
print(train)

将火车分割保存为JSONL格式

接下来，您将火车分割中的嵌入存储为JSONL格式的文件。每个嵌入都存储为：

{ 'id': .., 'embedding': [ ... ] }

索引的嵌入格式可以是CSV、JSON或Avro格式中的任一种。

了解有关[用于索引的嵌入格式](https://cloud.google.com/vertex-ai/docs/matching-engine/using-matching-engine#json)的更多信息。

In [ ]:
with open("glove100.json", "w") as f:
    for i in range(len(train)):
        f.write('{"id":"' + str(i) + '",')
        f.write('"embedding":[' + ",".join(str(x) for x in train[i]) + "]}")
        f.write("\n")

将以JSONL格式存储的嵌入存储在云存储中

接下来，您将上传训练数据到您的云存储桶。

In [ ]:
EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/matching_engine/initial/"
! gsutil cp glove100.json {EMBEDDINGS_INITIAL_URI}

创建向量搜索索引

接下来，您将为您的嵌入创建索引。目前支持两种索引算法：

- `create_tree_ah_index()`: 浅树 + 非对称哈希。
- `create_brute_force_index()`: 线性搜索。

在本教程中，您将使用`create_tree_ah_index()`来进行生产规模的工作。该方法带有以下参数：

- `display_name`: 索引的人类可读名称。
- `contents_delta_uri`: 嵌入的云存储位置，要么是要插入、更新或删除的嵌入。
- `dimensions`: 输入向量的维数。
- `approximate_neighbors_count`: (对于Tree AH) 在执行精确重新排序之前通过近似搜索找到的默认邻居数量。精确重新排序是通过更昂贵的距离计算重新排序通过近似搜索算法返回的结果的过程。
- `distance_measure_type`: 用于最近邻搜索的距离测量类型。
    - `SQUARED_L2_DISTANCE`: 欧几里德 (L2) 距离
    - `L1_DISTANCE`: 曼哈顿 (L1) 距离
    - `COSINE_DISTANCE`: 余弦距离。定义为1减去余弦相似度。
    - `DOT_PRODUCT_DISTANCE`: 默认值。定义为点积的负值。
- `description`: 索引的人类可读描述。
- `labels`: 以字典形式的用户元数据。
- `leaf_node_embedding_count`: 每个叶节点上的嵌入数量。如果未设置，默认值为1000。
- `leaf_nodes_to_search_percent`: 任何查询可能搜索的叶节点的百分比。必须在1-100的范围内，包括1和100。如果未设置，默认值为10 (表示10%)。

可能需要高达30分钟。

了解更多关于[配置向量搜索索引](https://cloud.google.com/vertex-ai/docs/matching-engine/configuring-indexes)。

In [ ]:
DIMENSIONS = 100
DISPLAY_NAME = "glove_100_1"

tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=EMBEDDINGS_INITIAL_URI,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    description="Glove 100 ANN index",
    labels={"label_name": "label_value"},
    # TreeAH specific parameters
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=7,
)

INDEX_RESOURCE_NAME = tree_ah_index.resource_name
print(INDEX_RESOURCE_NAME)

### 更新索引

接下来，您将使用新的嵌入更新索引 - 即插入。

#### 创建更新增量文件

首先，您需要制作一个包含要更新嵌入的 JSONL 文件。对于 `id` 为0的现有嵌入，您可以使用合成数据 - 在这种情况下，全部为零。然后，您将上传这个 JSONL 文件到云存储位置。

In [ ]:
with open("glove100_incremental.json", "w") as f:
    f.write(
        '{"id":"0","embedding":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}\n'
    )

EMBEDDINGS_UPDATE_URI = f"{BUCKET_URI}/matching-engine/incremental/"

! gsutil cp glove100_incremental.json {EMBEDDINGS_UPDATE_URI}

更新索引

接下来，您可以使用方法 `update_embeddings()` 来增量更新索引，具有以下参数：

- `contents_delta_uri`：嵌入的云存储位置，要么插入或更新。

可选地，参数 `is_complete_overwrite` 将替换整个索引。

In [ ]:
tree_ah_index = tree_ah_index.update_embeddings(
    contents_delta_uri=EMBEDDINGS_UPDATE_URI,
)

INDEX_RESOURCE_NAME = tree_ah_index.resource_name
print(INDEX_RESOURCE_NAME)

## 设置VPC对等连接网络

为了使用`矢量搜索索引`，您需要在您的项目和`Vertex AI矢量搜索`服务项目之间设置一个VPC对等连接网络。这可以消除网络流量中的额外跳跃，并允许使用高效的gRPC协议。

了解更多关于[VPC对等连接](https://cloud.google.com/vertex-ai/docs/general/vpc-peering)。

**重要提示：每个项目只能设置一个到`servicenetworking.googleapis.com`的VPC对等连接。**

### 为默认网络创建VPC对等连接

为了简化操作，我们为默认网络设置了VPC对等连接。您也可以为您的项目创建一个不同的网络。

如果您要与其他任何网络建立VPC对等连接，请确保该网络已经存在，并且您的虚拟机正在该网络上运行。

In [ ]:
# This is for display only; you can name the range anything.
PEERING_RANGE_NAME = "vertex-ai-prediction-peering-range"
NETWORK = "default"

# NOTE: `prefix-length=16` means a CIDR block with mask /16 will be
# reserved for use by Google services, such as Vertex AI.
! gcloud compute addresses create $PEERING_RANGE_NAME \
  --global \
  --prefix-length=16 \
  --description="peering range for Google service" \
  --network=$NETWORK \
  --purpose=VPC_PEERING

创建VPC连接

接下来，创建VPC对等连接。

*备注:* 如果收到`权限被拒绝`的提示，可能是您的默认服务帐户没有设置必要的角色`计算网络管理员`。在云控制台中，执行以下步骤。

1. 进入 `IAM & 管理员`
2. 找到您的服务帐户。
3. 点击编辑图标。
4. 选择 `添加另一个角色`。
5. 输入 `计算网络管理员`。
6. 选择 `保存`。

In [ ]:
! gcloud services vpc-peerings connect \
  --service=servicenetworking.googleapis.com \
  --network=$NETWORK \
  --ranges=$PEERING_RANGE_NAME \
  --project=$PROJECT_ID

检查您的对等连接状态。

In [ ]:
! gcloud compute networks peerings list --network $NETWORK

构建完整的网络名称

在随后为VPC对等连接创建`Vector Search Index Endpoint`资源时，您需要拥有完整的网络资源名称。

In [ ]:
full_network_name = f"projects/{PROJECT_NUMBER}/global/networks/{NETWORK}"

### 使用VPC网络创建索引终端点

接下来，您会创建一个 `矢量搜索索引终端点`，类似于使用点对点网络为预测创建 `私有终端点` 的概念。

要创建 `索引终端点` 资源，您需要使用以下参数调用 `create()` 方法：

- `display_name`：为 `索引终端点` 指定一个易于理解的名称。
- `description`：为 `索引终端点` 添加描述。
- `network`：VPC网络资源的名称。

In [ ]:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="index_endpoint_for_demo",
    description="index endpoint description",
    network=full_network_name,
)

INDEX_ENDPOINT_NAME = index_endpoint.resource_name
print(INDEX_ENDPOINT_NAME)

### 部署 `Vector Search Index` 到 `Index Endpoint` 资源

接下来，使用方法 `deploy_index()` 和以下参数将您的索引部署到 `Index Endpoint`:

- `display_name`: 已部署索引的人类可读名称。
- `index`: 您的索引。
- `deployed_index_id`: 已部署索引的用户分配标识符。
- `machine_type`: （可选）VM实例类型。
- `min_replica_count`: （可选）自动扩展的最小VM实例数。
- `max_replica_count`: （可选）自动扩展的最大VM实例数。

了解更多关于[Index Endpoint 的机器资源](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.indexEndpoints#DeployedIndex)

In [ ]:
DEPLOYED_INDEX_ID = "tree_ah_glove_deployed_" + TIMESTAMP

MIN_NODES = 1
MAX_NODES = 2
DEPLOY_COMPUTE = "n1-standard-16"

index_endpoint.deploy_index(
    display_name="deployed_index_for_demo",
    index=tree_ah_index,
    deployed_index_id=DEPLOYED_INDEX_ID,
    # machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

print(index_endpoint.deployed_indexes)

### 创建并执行在线查询

现在您的索引已部署，您可以进行查询。

首先，您使用合成数据构建一个向量 `query`，作为返回匹配项示例的示例。

接下来，您使用方法 `match()` 发送匹配请求，具有以下参数：

- `deployed_index_id`：已部署索引的标识符。
- `queries`：查询（实例）的列表。
- `num_neighbors`：要返回最接近匹配项的数量。

In [ ]:
# The number of nearest neighbors to be retrieved from database for each query.
NUM_NEIGHBOURS = 10

# Test query
queries = [
    [
        -0.11333,
        0.48402,
        0.090771,
        -0.22439,
        0.034206,
        -0.55831,
        0.041849,
        -0.53573,
        0.18809,
        -0.58722,
        0.015313,
        -0.014555,
        0.80842,
        -0.038519,
        0.75348,
        0.70502,
        -0.17863,
        0.3222,
        0.67575,
        0.67198,
        0.26044,
        0.4187,
        -0.34122,
        0.2286,
        -0.53529,
        1.2582,
        -0.091543,
        0.19716,
        -0.037454,
        -0.3336,
        0.31399,
        0.36488,
        0.71263,
        0.1307,
        -0.24654,
        -0.52445,
        -0.036091,
        0.55068,
        0.10017,
        0.48095,
        0.71104,
        -0.053462,
        0.22325,
        0.30917,
        -0.39926,
        0.036634,
        -0.35431,
        -0.42795,
        0.46444,
        0.25586,
        0.68257,
        -0.20821,
        0.38433,
        0.055773,
        -0.2539,
        -0.20804,
        0.52522,
        -0.11399,
        -0.3253,
        -0.44104,
        0.17528,
        0.62255,
        0.50237,
        -0.7607,
        -0.071786,
        0.0080131,
        -0.13286,
        0.50097,
        0.18824,
        -0.54722,
        -0.42664,
        0.4292,
        0.14877,
        -0.0072514,
        -0.16484,
        -0.059798,
        0.9895,
        -0.61738,
        0.054169,
        0.48424,
        -0.35084,
        -0.27053,
        0.37829,
        0.11503,
        -0.39613,
        0.24266,
        0.39147,
        -0.075256,
        0.65093,
        -0.20822,
        -0.17456,
        0.53571,
        -0.16537,
        0.13582,
        -0.56016,
        0.016964,
        0.1277,
        0.94071,
        -0.22608,
        -0.021106,
    ],
    [
        -0.99544,
        -2.3651,
        -0.24332,
        -1.0321,
        0.42052,
        -1.1817,
        -0.16451,
        -1.683,
        0.49673,
        -0.27258,
        -0.025397,
        0.34188,
        1.5523,
        1.3532,
        0.33297,
        -0.0056677,
        -0.76525,
        0.49587,
        1.2211,
        0.83394,
        -0.20031,
        -0.59657,
        0.38485,
        -0.23487,
        -1.0725,
        0.95856,
        0.16161,
        -1.2496,
        1.6751,
        0.73899,
        0.051347,
        -0.42702,
        0.16257,
        -0.16772,
        0.40146,
        0.29837,
        0.96204,
        -0.36232,
        -0.47848,
        0.78278,
        0.14834,
        1.3407,
        0.47834,
        -0.39083,
        -1.037,
        -0.24643,
        -0.75841,
        0.7669,
        -0.37363,
        0.52741,
        0.018563,
        -0.51301,
        0.97674,
        0.55232,
        1.1584,
        0.73715,
        1.3055,
        -0.44743,
        -0.15961,
        0.85006,
        -0.34092,
        -0.67667,
        0.2317,
        1.5582,
        1.2308,
        -0.62213,
        -0.032801,
        0.1206,
        -0.25899,
        -0.02756,
        -0.52814,
        -0.93523,
        0.58434,
        -0.24799,
        0.37692,
        0.86527,
        0.069626,
        1.3096,
        0.29975,
        -1.3651,
        -0.32048,
        -0.13741,
        0.33329,
        -1.9113,
        -0.60222,
        -0.23921,
        0.12664,
        -0.47961,
        -0.89531,
        0.62054,
        0.40869,
        -0.08503,
        0.6413,
        -0.84044,
        -0.74325,
        -0.19426,
        0.098722,
        0.32648,
        -0.67621,
        -0.62692,
    ],
]

matches = index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID, queries=queries, num_neighbors=NUM_NEIGHBOURS
)

for instance in matches:
    print("INSTANCE")
    for match in instance:
        print(match)

## 为校准创建暴力索引

暴力索引使用天真的暴力方法来寻找最近邻居。这种方法使用线性搜索，因此对于大规模索引来说并不高效。我们建议使用暴力索引来校准近似最近邻（ANN）索引以提高召回率，或者用于关键任务的匹配。

### 创建暴力索引

现在使用`create_brute_force_index()`方法来创建暴力索引。

为了确保苹果与苹果之间的比较，暴力索引的距离度量类型和特征规范类型，维度应与正在调整的生产指标相匹配。

In [ ]:
brute_force_index = aiplatform.MatchingEngineIndex.create_brute_force_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=EMBEDDINGS_INITIAL_URI,
    dimensions=DIMENSIONS,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    description="Glove 100 index (brute force)",
    labels={"label_name": "label_value"},
)

INDEX_BRUTE_FORCE_RESOURCE_NAME = brute_force_index.resource_name
print(INDEX_BRUTE_FORCE_RESOURCE_NAME)

更新索引

为了进行苹果与苹果的比较，您要对原始力量索引执行与 Tree AH 索引相同的增量更新。

In [ ]:
brute_force_index = tree_ah_index.update_embeddings(
    contents_delta_uri=EMBEDDINGS_UPDATE_URI
)

### 部署暴力索引到`IndexEndpoint`资源

接下来，您将暴力索引部署到同一个`IndexEndpoint`。

*注意:* 您可以将多个索引部署到同一个`Index Endpoint`资源。

In [ ]:
DEPLOYED_BRUTE_FORCE_INDEX_ID = "glove_brute_force_deployed_" + TIMESTAMP

index_endpoint.deploy_index(
    index=brute_force_index, deployed_index_id=DEPLOYED_BRUTE_FORCE_INDEX_ID
)

## 校准

现在你已经准备好进行校准。指数的生产版本采用了一种近似方法，这意味着与更慢的精确匹配（蛮力）方法相比，可能具有不完美的召回率。

### 获取两个索引的测试结果

首先，使用GloVe测试嵌入，您对两个指数进行相同的请求。

In [ ]:
prod_matches = index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=list(test),
    num_neighbors=NUM_NEIGHBOURS,
)

exact_matches = index_endpoint.match(
    deployed_index_id=DEPLOYED_BRUTE_FORCE_INDEX_ID,
    queries=list(test),
    num_neighbors=NUM_NEIGHBOURS,
)

### 计算召回率

最后，您可以根据结果确定从生产指数中召回的完全匹配的百分比。您随后可以使用这些信息来调整生产指数的部署。

In [ ]:
# Calculate recall by determining how many neighbors were correctly retrieved as compared to the brute-force option.
correct_neighbors = 0
for tree_ah_neighbors, brute_force_neighbors in zip(prod_matches, exact_matches):
    tree_ah_neighbor_ids = [neighbor.id for neighbor in tree_ah_neighbors]
    brute_force_neighbor_ids = [neighbor.id for neighbor in brute_force_neighbors]

    correct_neighbors += len(
        set(tree_ah_neighbor_ids).intersection(brute_force_neighbor_ids)
    )

recall = correct_neighbors / (len(test) * NUM_NEIGHBOURS)

print("Recall: {}".format(recall))

清理

要清理这个项目中使用的所有Google Cloud资源，您可以[删除您用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。
您还可以通过运行以下代码手动删除您创建的资源。

In [ ]:
# Force undeployment of indexes and delete endpoint
try:
    index_endpoint.delete(force=True)
except Exception as e:
    print(e)

# Delete indexes
try:
    tree_ah_index.delete()
    brute_force_index.delete()
except Exception as e:
    print(e)

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}